In [34]:
import numpy as np
import pandas as pd
import nltk
import re
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dense
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [31]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bot\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bot\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\bot\AppData\Roaming\nltk_data...


True

In [17]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [19]:
df_train["labels"] = df_train["malignant"].map({0: "Not Offensive",
                                                1: "Offensive" })

In [20]:
df_train = df_train[['comment_text', 'labels']]

In [21]:
df_train

,comment_text,labels
0,Explanation\nWhy the edits made under my usern...,Not Offensive
1,D'aww! He matches this background colour I'm s...,Not Offensive
2,"Hey man, I'm really not trying to edit war. It...",Not Offensive
3,"""\nMore\nI can't make any real suggestions on ...",Not Offensive
4,"You, sir, are my hero. Any chance you remember...",Not Offensive
...,...,...
159566,""":::::And for the second time of asking, when ...",Not Offensive
159567,You should be ashamed of yourself \n\nThat is ...,Not Offensive
159568,"Spitzer \n\nUmm, theres no actual article for ...",Not Offensive
159569,And it looks like it was actually you who put ...,Not Offensive


In [29]:
# cleaing the datasets
def cleaning(data):
    absolve_list = []
    
    for absolve in data:
        absolve = re.sub("[^a-zA-Z]"," ",absolve)
        absolve = absolve.lower()   # converting from uppercase to lowercase
        absolve = nltk.word_tokenize(absolve)
        lemma = nltk.WordNetLemmatizer()
        absolve = [ lemma.lemmatize(word) for word in absolve]
        absolve = " ".join(absolve)
        absolve_list.append(absolve)
        
    return absolve_list

In [33]:
df_train["comment_text"] = cleaning(df_train.comment_text)

df_test['new_comment_text'] = cleaning(df_test.comment_text)

In [36]:
x = np.array(df_train["comment_text"])
y = np.array(df_train["labels"])

In [37]:
cv = CountVectorizer()
X = cv.fit_transform(x) 

In [ ]:
X

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2]:
embeddings_dict = {}
with open("glove.6B.50d.txt", 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector
        

In [3]:
text = "Hello, how are you doing today bitch?"

In [9]:
tokens = nltk.word_tokenize(text)


In [12]:
embedding_dim = 50 # Dimensionality of GloVe embeddings
vocab_size = len(embeddings_dict)
tokenizer = Tokenizer()
embedding_matrix = np.zeros((vocab_size, embedding_dim))  # Initialize embedding matrix

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [13]:
max_sequence_length = 30
# Input token length

In [14]:
embedding_layer = Embedding(vocab_size, embedding_dim, weights=[embedding_matrix],
                            input_length=max_sequence_length, trainable=False)

In [40]:
from sklearn.tree import DecisionTreeClassifier

In [41]:
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
print("decision tree değeri: ", clf.score(X_test,y_test))

decision tree değeri:  0.9443835187216043


In [43]:
import tensorflow as tf

In [42]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

InvalidArgumentError: {{function_node __wrapped__SerializeManySparse_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[1] = [0,69600] is out of order. Many sparse ops require sorted indices.
    Use `tf.sparse.reorder` to create a correctly ordered copy.

 [Op:SerializeManySparse]